## 使用python部署so文件

### 1.导包

In [1]:
# 使用python部署*.so文件
import onnx
import time
import tvm
import numpy as np
import tvm.relay as relay
from PIL import Image
from tvm.contrib import graph_runtime



### 2.Load a test image

In [2]:
# ------------ load image
img = Image.open("street.jpg")
img = img.resize((416, 416))
img = np.array(img, dtype = np.float32)
img /= 255.0
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)

### 3.load model

In [5]:

libpath = "../models/yolov4_pc.so"
graph_json_path = "../models/yolov4_pc.json"
param_path = "../models/yolov4_pc.params"

# 接下来我们加载导出的模型去测试导出的模型是否可以正常工作
loaded_json = open(graph_json_path).read()
loaded_lib = tvm.runtime.load_module(libpath)
loaded_params = bytearray(open(param_path, "rb").read())

# 这里执行的平台为CPU
ctx = tvm.cpu()


module = graph_runtime.create(loaded_json, loaded_lib, ctx)
module.load_params(loaded_params)
module.set_input("input_0", img)
module.run()
out_deploy = module.get_output(0).asnumpy()
print(type(out_deploy))
print(out_deploy)


<class 'numpy.ndarray'>
[[[[ 1.58987451e+00  3.62838894e-01  1.57809094e-01 ...  1.97978824e-01
    -5.12179792e-01 -1.56389654e+00]
   [ 1.19889593e+00  4.58475292e-01 -2.45799392e-01 ...  1.08984329e-01
    -1.44762278e-01 -1.21538377e+00]
   [ 1.01471317e+00  2.55160689e-01  1.79528035e-02 ...  1.37108192e-03
    -1.23323761e-01 -1.07856166e+00]
   ...
   [ 6.41317964e-01 -1.04248846e+00 -4.72624823e-02 ... -4.59733874e-01
     8.08377564e-02 -1.12445247e+00]
   [ 7.29420960e-01 -1.29384112e+00 -6.04176164e-01 ... -5.29078603e-01
     2.25028098e-01 -1.40401328e+00]
   [ 1.61843908e+00  8.76458108e-01  3.72443527e-01 ... -1.08173966e+00
    -5.71893215e-01 -1.38488829e+00]]

  [[ 1.38944185e+00  1.25849736e+00  1.11148345e+00 ...  5.82397580e-01
     1.07248640e+00  1.79491627e+00]
   [ 4.71858889e-01  3.65642786e-01  4.94348109e-01 ... -1.01142041e-02
     2.17005517e-02  4.31442648e-01]
   [ 2.92619228e-01  2.63497382e-01  6.96207643e-01 ...  2.31302276e-01
     5.58940433e-02  1.

In [6]:
# 输出tvm运行的时间
since = time.time()
for i in range(100):
    module.run()
time_elapsed = time.time() - since
print('Time elapsed is {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Time elapsed is 0m 38s
